In [1]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jun  5 11:04:24 2019

@author: hove
"""

# run from D:\MyProjects\PR3916_Vincent\Werkmap\Python\Stap3\ReadMetaData

import hkvfewspy
import pandas as pd
from datetime import datetime
import numpy as np
import os
import netCDF4
from pyproj import Proj, transform
from netcdf_thredds.ReadMetaData_adjusted_v3 import ReadMetaData_adjusted

In [2]:

#%% 

### Dictionary of AQUO standard 

# OPAQ not in AQUO standard

aquo = {'C_0':
            {'grootheid' : 'GELDHD',
             'omschrijving': 'geleidendheid',
             'description': 'conductivity',
             'definitie': 'een maat voor het vermogen van een stof (vast, vloeibaar, gas of in oplossing) om de elektrische stroom te geleiden',
             'definition': 'a measure of the ability of a substance fluid (solid, fluid, gass or dissolved) to conduct an electric current'
             },
        'Chl_0':
            {'grootheid': 'CHLF',
             'omschrijving': '(massa) concentratie chlorofyl-a',
             'description': '(mass) concentration chlorophyll-a',
             'definitie': '(massa) concentratie van een groep groene pigmenten die het voor planten mogelijk maken fotosynthese uit te voeren. Chlorofyl-a is een indicator voor algen/fytoplankton',
             'definition': '(mass) concentration of a group of green pigments that enables plants to carry out photosynthesis. Chlorophyll-a is an indicator for algae/phytoplankton'
             },
        'OPAQ_0':
            {'grootheid': 'TRB',
             'omschrijving': 'turbiditeit',
             'description': 'turbidity',
             'definitie': 'een maat voor de troebelheid van een vloeistof',
             'definition': 'a measure of the turbidity of a fluid'
            },
        'Ph_0':
            {'grootheid': 'pH',
             'omschrijving': 'zuurgraad',
             'description': 'acidity', 
             'definitie': 'de negatieve waarde van de logaritme van de relatieve concentratie van H30+',     
             'definition': 'the negative value of the logarithm of the relative concentration of H3O+'
            },
        'T_0':
            {'grootheid': 'T',
             'omschrijving': 'water temperatuur',
             'description': 'water temperature', 
             'definitie': 'een maat voor hoe koud of warm het water is',
             'definition': 'a measure of how cold or warm the water is'
            }
        }

#%%

### Dictionary ###

framework = {'MPP': ['C_0', 'Chl_0', 'OPAQ_0', 'Ph_0', 'T_0']}

### Input variables ###

instrument = 'MPP'
location = 'FL67'
year = 2019
month = 4
day = 10

### Select times ###

startTime = pd.datetime(year, month, day)
endTime = startTime + pd.Timedelta(days=1)

#%%

### Get data from FEWS ###

pi = hkvfewspy.Pi(protocol='rest') 
url = 'https://db.dmhoutribdijk.nl/FewsWebServices/rest/fewspiservice/v1/'
pi.setUrl(url=url)

# Initiate and fill query object

filters = pi.getFilters()
filter_id = filters['f_ruw_ongevalideerd_{}'.format(instrument)]['id'] # 'f_ruw_ongevalideerd.{instrument}'

parameters = pi.getParameters()
parameter_id = parameters.loc['id', framework[instrument]].tolist()

locations = pi.getLocations()
location_id = locations[location]['locationId']

query_parameters = pi.setQueryParameters(protocol='rest')

query_parameters.filterId(filter_id)
query_parameters.parameterIds(parameter_id)
query_parameters.locationIds(location_id)
query_parameters.startTime(startTime)
query_parameters.endTime(endTime)
query_parameters.useMilliseconds(True) 
query_parameters.version('1.25')

timeseries = pi.getTimeSeries(query_parameters, print_response=False)

# Pivot dataframe to keep time as index but to get parameterIds as columns
df = timeseries.pivot(columns='parameterId', values='value')

In [3]:
delta = pd.timedelta_range(start=0, periods=144, freq='ms')
df.index = df.index + delta

In [8]:
df.index

DatetimeIndex([       '2019-04-10 00:09:00+00:00',
               '2019-04-10 00:19:00.001000+00:00',
               '2019-04-10 00:29:00.002000+00:00',
               '2019-04-10 00:39:00.003000+00:00',
               '2019-04-10 00:49:00.004000+00:00',
               '2019-04-10 00:59:00.005000+00:00',
               '2019-04-10 01:09:00.006000+00:00',
               '2019-04-10 01:19:00.007000+00:00',
               '2019-04-10 01:29:00.008000+00:00',
               '2019-04-10 01:39:00.009000+00:00',
               ...
               '2019-04-10 22:29:00.134000+00:00',
               '2019-04-10 22:39:00.135000+00:00',
               '2019-04-10 22:49:00.136000+00:00',
               '2019-04-10 22:59:00.137000+00:00',
               '2019-04-10 23:09:00.138000+00:00',
               '2019-04-10 23:19:00.139000+00:00',
               '2019-04-10 23:29:00.140000+00:00',
               '2019-04-10 23:39:00.141000+00:00',
               '2019-04-10 23:49:00.142000+00:00',
            

In [4]:
#%% 

### Get metadata ###

file_metadata = r"D:\jupyter notebooks\3916.10 Houtribdijk\netcdf_thredds\Houtribdijk_OenM_Metadata-Meetpalen.xlsx"

ts = [startTime] # it cannot be a scalar value, so pass it as a list

read_meta_data = ReadMetaData_adjusted(file_metadata, ts)
meta_data = read_meta_data.get_meta_data()

#%%

### X,Y and lat,lon ###

inProj = Proj(init='epsg:7415') # X,Y
outProj = Proj(init='epsg:4326') # Lon,Lat

X = meta_data[location]['X'][0] 
Y = meta_data[location]['Y'][0]

lon, lat = transform(inProj, outProj, X, Y)

#%%

### Write to netcdf ###

ncdir = r'D:\jupyter notebooks\3916.10 Houtribdijk\netcdf_thredds\Stap4_NetCDF\{}\{}'.format(instrument, location)
ncfile = '{}_{}_{}{}{}_new.nc'.format(instrument, location, year, str(month).zfill(2), str(day).zfill(2))
filename = os.path.join(ncdir, ncfile)

In [5]:
filename

'D:\\jupyter notebooks\\3916.10 Houtribdijk\\netcdf_thredds\\Stap4_NetCDF\\MPP\\FL67\\MPP_FL67_20190410_new.nc'

In [6]:
with netCDF4.Dataset(filename, mode='w') as nc:

    ### Add geodata ###  
    
    # 1) Latitude
    
    nc.createDimension('latitude', 1)
    nc.createVariable('latitude', 'f4', (u'latitude',))
    # data
    nc.variables['latitude'][:] = lat
    # attributes
    nc.variables['latitude'].long_name = 'measurement location latitude'
    nc.variables['latitude'].standard_name = 'latitude'
    nc.variables['latitude'].units = 'degrees_north'
    nc.variables['latitude'].grid_mapping = 'wgs84'
    nc.variables['latitude'].axis = 'Y'
    
    # 2) Longitude
    
    nc.createDimension('longitude', 1)
    nc.createVariable('longitude', 'f4', (u'longitude',))
    # data
    nc.variables['longitude'][:] = lon
    # attributes
    nc.variables['longitude'].long_name = 'measurement location longitude'
    nc.variables['longitude'].standard_name = 'longitude'
    nc.variables['longitude'].units = 'degrees_east'
    nc.variables['longitude'].grid_mapping = 'wgs84'
    nc.variables['longitude'].axis = 'X'
    
    # 3) X
    
    nc.createDimension('X', 1)
    nc.createVariable('X', 'f4', (u'X',))
    # data
    nc.variables['X'][:] = X
    # attributes
    nc.variables['X'].long_name = 'measurement location x'
    nc.variables['X'].standard_name = 'projection_x_coordinate'
    nc.variables['X'].units = 'm'
    nc.variables['X'].grid_mapping = 'epsg'
    
    # 4) Y
    
    nc.createDimension('Y', 1)
    nc.createVariable('Y', 'f4', (u'Y',))
    # data
    nc.variables['Y'][:] = Y
    # attributes
    nc.variables['Y'].long_name = 'measurement location y'
    nc.variables['Y'].standard_name = 'projection_y_coordinate'
    nc.variables['Y'].units = 'm'
    nc.variables['Y'].grid_mapping = 'epsg'
    
    # 5) WGS84
    
    nc.createDimension('wgs84', 0)
    nc.createVariable('wgs84', 'f4', (u'wgs84',))
    # attributes
    nc.variables['wgs84'].standard_name = 'WGS 84'
    nc.variables['wgs84'].epsg = '4326'
    nc.variables['wgs84'].grid_mapping_name = 'latitude_longitude'
    nc.variables['wgs84'].semi_major_axis = '6.37814e+06'
    nc.variables['wgs84'].semi_minor_axis = '6.35675e+06'
    nc.variables['wgs84'].inverse_flattening = '298.257'
    nc.variables['wgs84'].proj4_params = "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs" 
    nc.variables['wgs84'].EPSG_code = 'EPSG:4326'
    nc.variables['wgs84'].projection_name = 'Latitude Longitude'
    nc.variables['wgs84'].wkt = ''
    nc.variables['wgs84'].comment = 'value is equal to EPSG code'
    
    # 6) EPSG
    
    nc.createDimension('epsg', 0)
    nc.createVariable('epsg', 'f4', (u'epsg',))
    # attributes
    nc.variables['epsg'].standard_name = 'Amersfoort / RD New'
    nc.variables['epsg'].epsg = '7415'
    nc.variables['epsg'].epsg_name = 'Oblique Stereographic'
    nc.variables['epsg'].grid_mapping_name = 'stereographic'
    nc.variables['epsg'].semi_major_axis = '6.3774e+06'
    nc.variables['epsg'].semi_minor_axis = '6.35608e+06'
    nc.variables['epsg'].inverse_flattening = '299.153'
    nc.variables['epsg'].latitude_of_projection_origin = '52.0922'
    nc.variables['epsg'].longitude_of_projection_origin = '5.23155'
    nc.variables['epsg'].false_easting = '155000'
    nc.variables['epsg'].false_northing = '463000'
    nc.variables['epsg'].scale_factor_at_projection_origin = '0.999908'
    nc.variables['epsg'].proj4_params = "+proj=sterea +lat_0=52.15616055555555 +lon_0=5.38763888888889 +k=0.999908 +x_0=155000 +y_0=463000 +ellps=bessel +units=m +towgs84=565.4174,50.3319,465.5542,-0.398957388243134,0.343987817378283,-1.87740163998045,4.0725 +no_defs" 
    nc.variables['epsg'].EPSG_code = 'EPSG:7415'
    nc.variables['epsg'].projection_name = 'Amersfoort / RD New'
    nc.variables['epsg'].wkt = ''
    nc.variables['epsg'].comment = 'value is equal to EPSG code' 

    # 6) Z (Z CAN HAVE LEVELS FOR OTHER INSTRUMENTS)

    nc.createDimension('Z', 1)
    nc.createVariable('Z', 'f4', (u'Z',))
    # data
    nc.variables['Z'][:] = meta_data[location]['{}_NAP_Z'.format(instrument)][0]
    # attributes
    nc.variables['Z'].long_name = 'instrument height at measurement location'
    nc.variables['Z'].standard_name = 'height'
    nc.variables['Z'].units = 'm+NAP'
    nc.variables['Z'].axis = 'Z'
    
    ### Add time ###
    
    nc.createDimension('time', len(df.index))
    
    nc.createVariable('time', 'u8', (u'time',))
    
    # add attributes
    nc.variables['time'].short_name = 'time'
    nc.variables['time'].long_name = 'time of measurement'
    nc.variables['time'].units = 'milliseconds since 1970-01-01 00:00:00.000'
    nc.variables['time'].calendar = 'gregorian' # toch???

    #timedeltaFromOrigin = df.index - pd.Timestamp('1970-01-01 00:00:00.000', tz='Etc/GMT') 
    nc.variables['time'][:] = (df.index.astype(np.int64) // 10**6).values #timedeltaFromOrigin.total_seconds().values
    
    ### Add parameters ###
    
    for i, ix in enumerate(framework[instrument]):
    
        nc.createVariable(aquo[ix]['grootheid'], 'f4', (u'time'))
    
        # add attributes from dictionary based on aquo standard and from FEWS data
        if aquo[ix]['grootheid'] == 'CHLF':
            nc.variables[aquo[ix]['grootheid']].units = 'microgram/L'
        else:
            nc.variables[aquo[ix]['grootheid']].units = parameters[ix].unit  
        nc.variables[aquo[ix]['grootheid']].dutch_short_name = aquo[ix]['omschrijving']
        nc.variables[aquo[ix]['grootheid']].dutch_description = aquo[ix]['definitie']
        nc.variables[aquo[ix]['grootheid']].short_name = aquo[ix]['description']
        nc.variables[aquo[ix]['grootheid']].description = aquo[ix]['definition']
    
        nc.variables[aquo[ix]['grootheid']][:] = df[parameter_id[i]].values

    ### Add global info ###
    
    nc.summary = 'data measured by {} instrument at location {}'.format(instrument, location)

    nc.time_start = datetime.strftime(startTime, format='%Y-%m-%d %H:%M:%S.%f')
    nc.time_end = datetime.strftime(endTime, format='%Y-%m-%d %H:%M:%S.%f')
    nc.time_duration = str(endTime-startTime)
    nc.instrument = instrument
    nc.location = location
    
    nc.sync()

#%%    
### Read netcdf ###
    
dataset = netCDF4.Dataset(filename)

time_sec = dataset.variables['time'][:].data
time = pd.to_datetime(time_sec, unit='ms')

dataset.close()

In [9]:
time

DatetimeIndex([       '2019-04-10 00:09:00', '2019-04-10 00:19:00.001000',
               '2019-04-10 00:29:00.002000', '2019-04-10 00:39:00.003000',
               '2019-04-10 00:49:00.004000', '2019-04-10 00:59:00.005000',
               '2019-04-10 01:09:00.006000', '2019-04-10 01:19:00.007000',
               '2019-04-10 01:29:00.008000', '2019-04-10 01:39:00.009000',
               ...
               '2019-04-10 22:29:00.134000', '2019-04-10 22:39:00.135000',
               '2019-04-10 22:49:00.136000', '2019-04-10 22:59:00.137000',
               '2019-04-10 23:09:00.138000', '2019-04-10 23:19:00.139000',
               '2019-04-10 23:29:00.140000', '2019-04-10 23:39:00.141000',
               '2019-04-10 23:49:00.142000', '2019-04-10 23:59:00.143000'],
              dtype='datetime64[ns]', length=144, freq=None)